In [2]:
import os
import tensorflow as tf
import matplotlib
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets
from matplotlib import pyplot as plt

matplotlib.rcParams['font.size'] = 20
matplotlib.rcParams['figure.titlesize'] = 20
matplotlib.rcParams['figure.figsize'] = [9, 7]
matplotlib.rcParams['font.family'] = ['STKaiTi']
matplotlib.rcParams['axes.unicode_minus']=False 

# 数据读取和预处理
# 1. 读取训练集、测试集数据
# 2. 对数据进行预处理。X数据（像素灰度）进行归一化处理，一般处理到[0, 1]或[-1, 1]之间
#    对Y数据（分类标签）进行one-hot编码，去掉数字标签可能带来的大小关系
# 3. 使用tf.data.Dataset.from_tensor_slice((x, y)) 将三维的图片数据，按照第一个维度进行展开，及进行“打平”操作
# 4. 使用tf.data.Dataset.from_tensor_slice.batch 方法，设置批处理数据的大小
(x, y), (x_val, y_val) = tf.keras.datasets.mnist.load_data() #  如果没有from ... import ... 语句，需要一层层引用
x = tf.convert_to_tensor(x, dtype=tf.float32)/255. # 转换数据范围到[0,1]
y = tf.convert_to_tensor(y, dtype=tf.int32)
y = tf.one_hot(y, depth=10) # one-hot 编码，去掉标签的大小关系
print(x.shape, y.shape)
# tf.data.Dataset.from_tensor_slices真正作用是切分传入Tensor的第一个维度，生成相应的dataset，即第一维表明数据集中数据的数量，之后切分batch等操作都以第一维为基础。
# 打平后的数据维度为((28, 28)图片, (10, )标记)
train_dataset = tf.data.Dataset.from_tensor_slices((x, y)) 
print(train_dataset)
train_dataset = train_dataset.batch(600) # 暂：转换完的维度(None, 28, 28)中None
#train_dataset = train_dataset.repeat(20) # 如果用for epoch in range(20)，则不用这一句，效果一样
print(train_dataset)

(60000, 28, 28) (60000, 10)
<TensorSliceDataset shapes: ((28, 28), (10,)), types: (tf.float32, tf.float32)>
<BatchDataset shapes: ((None, 28, 28), (None, 10)), types: (tf.float32, tf.float32)>


In [3]:
# 网络搭建
# 1. 使用Sequential容器，搭建3曾网络如下
# 2. 需要乡下一层传递时，需使用activation函数
# 3. 输出层因不用向下一层传递，因此不用激活函数activation
model = keras.Sequential([ # 三个非线性层的嵌套模型，包括两个隐藏层，一个输出层
        layers.Dense(256, activation='relu'), # 隐藏层1，用relu作为激活函数
        layers.Dense(512, activation='relu'),
        layers.Dense(10)]) # 输出层不用激活函数，输出节点数为10

# 定义优化器
optimizer = optimizers.SGD(learning_rate=0.1) 

In [4]:
# 模型训练

losses = []
# ''' 用epoch和for循环实现迭代训练
for epoch in range(30):
    # 第四步： 循环迭代优化
    for step, (x, y) in enumerate(train_dataset): # python 语法，enumerate()方法将train_dataset中的后两维数据以枚举形式付给step对应的(x, y)变量
        with tf.GradientTape() as tape: # python 语法，对于后续需要释放的资源，使用with...as...语句，避免忘记释放资源
            # ？？？
            x = tf.reshape(x, (-1, 28*28)) # 打平操作
            out = model(x) # 第一步：得到模型输出, 相当于表达式中的Y
            # 第二步：计算平均误差
            loss = tf.square(out - y) # 计算平方和[b, 10]
            loss = tf.reduce_sum(loss) / x.shape[0] # 计算每个样本的平均误差[b]
            # 第三步：计算并优化参数[w1, w2, w3, b1, b2, b3]
            grads = tape.gradient(loss, model.trainable_variables) # 自动计算梯度
            optimizer.apply_gradients(zip(grads, model.trainable_variables)) # w'=w-lr*grad, 更新网络参数
        # 每500次计算绘制一次图像
        if step % 20 == 0: 
            print(epoch, step, 'loss:', loss.numpy())
        # print(epoch, step, 'loss:', loss.numpy()) # step = 数据第一维度数量（图片张数）/ n； 最大值为 batch(n) 中的n
    losses.append(float(loss))
# '''

'''另一种实现方式Dataset.repeat()   没搞懂！！！
    # 第四步： 循环迭代优化
for step, (x, y) in enumerate(train_dataset): 
    with tf.GradientTape() as tape: # python 语法，对于后续需要释放的资源，使用with...as...语句，避免忘记释放资源
        x = tf.reshape(x, (-1, 28*28)) # 打平操作
        out = model(x) # 第一步：得到模型输出, 相当于表达式中的Y
        # 第二步：计算平均误差
        loss = tf.square(out - y) # 计算平方和[b, 10]
        loss = tf.reduce_sum(loss) / x.shape[0] # 计算每个样本的平均误差[b]
        # 第三步：计算并优化参数[w1, w2, w3, b1, b2, b3]
        grads = tape.gradient(loss, model.trainable_variables) # 自动计算梯度
        optimizer.apply_gradients(zip(grads, model.trainable_variables)) # w'=w-lr*grad, 更新网络参数
        # 每500次计算绘制一次图像
#        if step % 100 == 0: 
#            print(epoch, step, 'loss:', loss.numpy())
    print(step, 'loss:', loss.numpy())
losses.append(float(loss))
'''

0 0 loss: 1.6420572
0 20 loss: 0.58835083
0 40 loss: 0.46164078
0 60 loss: 0.37137765
0 80 loss: 0.29022345
1 0 loss: 0.2672438
1 20 loss: 0.29224572
1 40 loss: 0.24534388
1 60 loss: 0.2422368
1 80 loss: 0.2075305
2 0 loss: 0.19581568
2 20 loss: 0.21875928
2 40 loss: 0.19606483
2 60 loss: 0.19903563
2 80 loss: 0.17718397
3 0 loss: 0.1662726
3 20 loss: 0.18863627
3 40 loss: 0.16992478
3 60 loss: 0.17534819
3 80 loss: 0.15844448
4 0 loss: 0.14903355
4 20 loss: 0.17009947
4 40 loss: 0.15294532
4 60 loss: 0.15936646
4 80 loss: 0.14518616
5 0 loss: 0.13706009
5 20 loss: 0.15673381
5 40 loss: 0.14061546
5 60 loss: 0.1472569
5 80 loss: 0.13465223
6 0 loss: 0.12803301
6 20 loss: 0.1463935
6 40 loss: 0.1309917
6 60 loss: 0.13777399
6 80 loss: 0.1260041
7 0 loss: 0.12079926
7 20 loss: 0.13784757
7 40 loss: 0.12301956
7 60 loss: 0.13025157
7 80 loss: 0.1187753
8 0 loss: 0.11473055
8 20 loss: 0.13069613
8 40 loss: 0.11634931
8 60 loss: 0.123904645
8 80 loss: 0.112533785
9 0 loss: 0.10954019
9 20 l

"另一种实现方式Dataset.repeat()   没搞懂！！！\n    # 第四步： 循环迭代优化\nfor step, (x, y) in enumerate(train_dataset): \n    with tf.GradientTape() as tape: # python 语法，对于后续需要释放的资源，使用with...as...语句，避免忘记释放资源\n        x = tf.reshape(x, (-1, 28*28)) # 打平操作\n        out = model(x) # 第一步：得到模型输出, 相当于表达式中的Y\n        # 第二步：计算平均误差\n        loss = tf.square(out - y) # 计算平方和[b, 10]\n        loss = tf.reduce_sum(loss) / x.shape[0] # 计算每个样本的平均误差[b]\n        # 第三步：计算并优化参数[w1, w2, w3, b1, b2, b3]\n        grads = tape.gradient(loss, model.trainable_variables) # 自动计算梯度\n        optimizer.apply_gradients(zip(grads, model.trainable_variables)) # w'=w-lr*grad, 更新网络参数\n        # 每500次计算绘制一次图像\n#        if step % 100 == 0: \n#            print(epoch, step, 'loss:', loss.numpy())\n    print(step, 'loss:', loss.numpy())\nlosses.append(float(loss))\n"

In [1]:
# 每个epoch次训练，绘制偏移量




plt.figure()
plt.plot(losses, color='C0', marker='s', label='训练')
plt.xlabel('Epoch')
plt.legend()
plt.ylabel('MSE')
plt.savefig('forward.svg')
plt.show()

NameError: name 'matplotlib' is not defined